<b><u><title>Redukcja wymiarów oraz selekcja cech kluczowych wielowymiarowych zbiorów dancyh.</title></b></u>

--------------------------------------------------------------------------

# Krótkie słowa wstępu.

Uprawianie Data Science w bioinformatyce oraz innych wymagających cierpliwości, specjalistycznej wiedzy oraz dotczących relanych problemów świata dziedzin nauki jest wciąż raczkującym podejściem. Rozumienie stosowanych metod niekiedy schodzi na dalszy plan co jest skutkiem goniących terminów oraz rzekomej złożoności matematyki stojącej za owymi rozwiązaniami.
Niekiedy metody te oparte na prostej statystyce, będąc zachwalane w publikacjach, tutorialach oraz na przestrzeniu forum internetowych nie powinny mieć zastosowania do większości danych biologicznych, lecz stanowią swoisty "trigger word", który ma za zadanie pokazać że badacz jest na czasie, "cool" oraz nie boi się interdyscyplinarności.


## Principal Component / Coordinate Analysis, czyli jak być fajnym ale nie koniecznie poprawnym.

Nie raz rozmawiając z osobami potencjalnie zajmującymi się podobnymi problemami badawczymi (biostatystycy, data scientists, biolodzy) natykałem się ze słowami rodem z pasty o [fanatyku wędkarstwa](https://copypasta.fandom.com/pl/wiki/M%C3%B3j_stary_to_fanatyk_w%C4%99dkarstwa.) "PCA jest król genomiki, tak jak lew jest król dżungli!". Jeśli słowa te pozostawiają u was cierpki wyraz twarzy, nie spojlerujcie mniej zaznajomionym kolegom dlaczego. Dzisiaj sami dojdą do tych wniosków.

## Podstawa użytkowania PCA

Jeśli wkleiłbym tutaj wzór na PCA, połowa z was by zasnęła, 1/4 się zaśmiała a 1/4 wyszła. Wklejanie tak skomplikowanych wzorów mija się z celem. Jeśli ktoś z was będzie chciał faktycznie zgłębić jego tajemnicę to [tutaj](https://www.youtube.com/watch?v=kw9R0nD69OU) oraz [tam](https://www.youtube.com/watch?v=rng04VJxUt4) znajdziecie ważniejsze informacje na temat jego działania. Moim dzisiejszym zadaniem jest przekazanie wam intuicji dotyczącej tego kiedy używać tego algorytmu a kiedy nie. Zatem do rzeczy, macie 5 minut żeby przypomnieć sobie kluczową różnicę pomiędzy testem parametrycznym takim jak ANOVA a nie parametrycznym takim jak Kruskal-Wallis test. Uścisk ręki prezesa temu kto najprościej powie jak sprawdzić kiedy stosować jeden test a kiedy drugi. Czas start! Odpalamy komórkę poniżej i lecimy z tematem!





In [ ]:
import time
for i in range(5):
  if i < 4:
    print("{} min left".format(5 - i))
    time.sleep(60)
  else:
    print("{} min left".format(5 - i))
    time.sleep(60)
    print("Czas na odpowieź")

# Sprawdźmy czy macie rację
Poniżej zaimportujemy pewien zestaw danych.
Korzystając z dokumentacji [`sklearn`](https://scikit-learn.org/stable/) i [`scipy`](https://www.scipy.org/docs.html) będziecie dziś na nim operować, tak aby stworzyć najlepszy możliwy model prostej regresji logistycznej.

Dane są wielowymiarowe i należy wyciągnąć z nich jedyne przydatne informacje do klasyfikacji.

Dla utrudnienia użyjemy datasetu Wisconsin Breast Cancer bez nazw kolumn

In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data & wget http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names

In [ ]:
# Importujemy bibliotekę pandas by móc działać na data frames
import pandas

# Wczytujemy plik csv z usunięciem informacji o nazwach kolumn
data = pandas.read_csv("wdbc.data", header = None)
cols = data.columns.to_list()

# Nadajemy "dummy" nazwy kolumn
for i in range(len(cols)):
  if i == 0:
    cols[i] = "Index"
  elif i == 1:
    cols[i] = "Tumor"
  else:
    cols[i] = "Feature {}".format(i-1)
data.columns = cols

# Czyścimy co nieco kolumnę kategorii
data.Tumor = data.Tumor.replace("M", "Malignant")
data.Tumor = data.Tumor.replace("B", "Benign")
column_idx_names = data.columns

# Usuwamy ID pacjenta i przywracamy nazwy kolumn zgodne z indeksami
data = data.iloc[:,1:]

# Wyświetlamy pierwsze 10 rzędów data frame'u
data.head(10)

In [ ]:
# Printujemy wartości unikalne kolumny zawierającej kategorie
print(data.Tumor.unique())

In [ ]:
# Metadane, nie obowiązkowe do zapoznania się
with open("wdbc.names","rb") as fh:
  for line in fh.readlines():
    print(line)

## Importowanie bibliotek
Korzystając z dobrodziejstw Google Collab zaimportujemy biblioteki bez których bylibyśmy jak pijane dziecko we mgle.


In [ ]:
import pandas
from scipy import stats
import itertools
from sklearn import preprocessing, linear_model, model_selection, metrics
from sklearn import decomposition, ensemble, feature_selection
from sklearn.manifold import MDS
import numpy
from matplotlib import pyplot
import operator
%matplotlib inline

### **Zadanie 1.**
Sprawdź rozkład kolumn z datasetu WBC przy użyciu odpowiedniej funkcji z biblioteki `scipy.stats` (punkt za umiejętność korzystania z dokumentacji)

## Tworzenie funkcji do sprawdzenia rozkładu zmiennych dla każdej z obecnych klas

In [ ]:
# Uzupełnij kod funkcji przedstawionej poniżej

def StratifiedDistributionCheck(dataframe, list_of_tumors):
  for tumor in list_of_tumors:
    df = dataframe[dataframe.Tumor == tumor]
    print("Wyniki dla {} tumor type\n -----------------------\n".format(tumor))
    for column in df.columns:
      if column == "<NAZWA_KOLUMNY_DO_POMINIĘCIA>":
        None
      else:
        p = stats.<NAZWA_FUNKCJI>(df.loc[:,column])[1]
        print("{0:s} <NAZWA_TESTU> test p-value wynosi {1:.3f}\n".format(
            column,p))
        if p > 0.05:
          print("Twoje wytłumaczenie wyniku działania konstrukcji warunkowej")
        else:
          print("Twoje wytłumaczenie wyniku działania konstrukcji warunkowej")

StratifiedDistributionCheck(data,data.Tumor.unique())

### **Zadanie 2.**
Wytłumacz co oznaczają powyższe wyniki (punkt za intuicję)


# Data Science w praktyce
W tej części zobaczycie (i mam nadzieję też zrozumiecie, co z czym się je)

## Definiowanie modelu regresji logistycznej z użyciem `sklearn`
Stworzymy teraz funkcje zwracające model oraz wynik działania modelu na danych wejściowych, tak aby móc porównać wpływ różnych metod redukcji wymiarów na dane (oraz zobaczyć jak przebiega klasyfikacja w przypadku oryginalnych danych).

Najpierw jednak rozbijemy DataFrame na 2 macierze, zawierające dane numeryczne, potrzebne do wytrenowania i testowania modelu, oraz zawierające binarnie enkodowane typy guza.

In [ ]:
X = data.iloc[:,1:].values
Y_str = data.iloc[:,0]
lb = preprocessing.LabelBinarizer()
lb.fit(Y_str)
print("Typy guzów do binaryzacji\n {} = 0\n {} = 1".format(lb.classes_[0],
                                                               lb.classes_[1]))
Y = lb.transform(Y_str)
print("\nRozmiar matrycy X\n{}".format(X.shape))
print("\nPierwsze dwa rzędy X\n{}".format(X[:2,:]))
print("\nRozmiar matrycy Y\n{}".format(Y.shape))
print("\nPierwsze dwa typy guzów Y\n{}".format(Y[:2,:]))


## Sztuczne zwiększenie wymiarowości danych - nie róbcie tego w domu
Jako że jest to jeden z prostszych datasetów do nauki ML to żeby nie było za łatwo, dodamy randomowy szum gaussowski i nieparametryczny do danych, tak aby sztucznie zwiększyć ich wymiarowość


In [ ]:
def AddArtificialData(X, random_state = 2137, art_columns = X.shape[1]):
  numpy.random.seed(seed = random_state)
  X_artif = numpy.zeros(shape=(X.shape[0], X.shape[1]+art_columns))
  for i in range(X_artif.shape[1]):
    condition1 = numpy.random.randint(0,2)
    if condition1 == 0:
      X_artif[:,i] = numpy.random.normal(numpy.random.randint(20,90),numpy.random.randint(5,20),)
    else:
      condition2 = numpy.random.randint(0,2)
      if condition2 == 0:
        X_artif[:,i] = numpy.random.rayleigh(scale = numpy.random.randint(1,90),
                                             size = X.shape[0])
      else:
        X_artif[:,i] = numpy.random.rand(X_artif.shape[0])
  
  X_artif[:,:X.shape[1]] = X
  return X_artif


In [ ]:
X = AddArtificialData(X, random_state = 90)

Poniższa funkcja zwracać będzie gotowy wytrenowany model, wynik działania tego modelu oraz metryki do późniejszego benchmarku

In [ ]:
def LogitModel(X, Y, penalty = "l1", reg = 0.2, solver = "liblinear",
               random_state = 2137, n_iter = 100, split = 0.7, verbose = 1):
  Y = Y.ravel()
  c = 1 - reg
  s = 1 - split
  clf = linear_model.LogisticRegression(penalty = penalty,
                                        C = 1-reg,
                                        solver = solver,
                                        random_state = random_state,
                                        max_iter = n_iter,
                                        verbose = verbose,
                                        )
  X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y,
                                                                      test_size = s,
                                                                      random_state = random_state)
  
  clf = clf.fit(X_train, Y_train)
  Y_pred = clf.predict(X_test)
  scores = {}
  scores["Mean Accuracy"] = clf.score(X_test, Y_test)
  scores["F1"] = metrics.f1_score(Y_test, Y_pred)
  scores["AUC"] = metrics.roc_auc_score(Y_test, Y_pred)
  scores["Confusion Matrix"] = metrics.confusion_matrix(Y_test, Y_pred)
  scores["ROC Curve"] = metrics.roc_curve(Y_test, Y_pred)
  
  pyplot.matshow(scores["Confusion Matrix"])
  pyplot.title("Confusion Matrix")
  pyplot.colorbar()
  pyplot.ylabel("True label")
  pyplot.xlabel("Predicted Label")
  pyplot.show()

  fpr, tpr, _ = metrics.roc_curve(Y_test.ravel(), Y_pred.ravel())
  roc_auc = metrics.auc(fpr, tpr)

  pyplot.figure()
  lw = 2
  pyplot.plot(fpr, tpr, color='darkorange',
          lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
  pyplot.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  pyplot.xlim([0.0, 1.0])
  pyplot.ylim([0.0, 1.05])
  pyplot.xlabel('False Positive Rate')
  pyplot.ylabel('True Positive Rate')
  pyplot.title('Receiver operating characteristic example')
  pyplot.legend(loc="lower right")
  pyplot.show()


  for i, v in scores.items():
    if i == "Confusion Matrix":
      print("{}\n{}".format(i,v))
    elif i == "ROC Curve":
      None
    else:
      print("{0:s} metic has value of {1:.3f}".format(i,v))
  return (clf, Y_test, Y_pred, scores)
  


## Redukcja wymiarowości danych

### **Zadanie 3**

Zastosuj [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), [MDS](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html), [Feature Selection bazujące na drzewach decyzyjnych](https://scikit-learn.org/stable/modules/feature_selection.html#tree-based-feature-selection) oraz Infinite Feature Selection na danych wejściowych. Empirycznie dobierz najlepsze hiperparametry w miejscach wskazanych w kodzie. (punkt za eksperymentowanie z kodem)


<u>Dla przypomnienia - wymiarowość pełnego zestawu danych</u>


In [ ]:
X.shape[1]

#### Infinite Feature Selection - czyli coś nietypowego

obraz.png

In [ ]:
def InfiniteFeatureSelection(X, n_features, alpha):
  all_permutations = list(itertools.permutations(range(X.shape[1]), 2))
  importance_scores = []
  max_var_idx = []
  feature_idx = list(range(n_features))
  X_important = numpy.zeros(shape = (X.shape[0], n_features))
  
  for element in all_permutations:
    element1 = element[0]
    element2 = element[1]
    v1 = numpy.var(X[:,element1])
    v2 = numpy.var(X[:,element2])

    max_var = max(v1,v2)
    if max_var == v1:
      max_var_idx.append(str(element1))
    else:
      max_var_idx.append(str(element2))
    
    spearman_coef, p = stats.spearmanr(X[:,element1], X[:,element2])
    reg_max_var = numpy.multiply(alpha,max_var)
    reg_spearman_based_coef = numpy.multiply((1 - alpha),
                                          (1 - numpy.abs(spearman_coef)))
    importance_score =  reg_max_var + reg_spearman_based_coef
    importance_scores.append(importance_score)
  
  data = {"Max_var_idx":max_var_idx, "Importance_scores":importance_scores}
  importance_df = pandas.DataFrame(data = data)
  sum_scores = {}
  for unq in importance_df.Max_var_idx.unique():
    temp_df = importance_df.loc[importance_df.Max_var_idx == unq,:]
    colsum = temp_df.Importance_scores.sum(axis=0)
    sum_scores[str(unq)] = colsum
  sorted_scores = sorted(sum_scores.items(), key=operator.itemgetter(1),
                         reverse = True)
  most_important_features = list(operator.itemgetter(*feature_idx)(sorted_scores))
  for i in range(n_features):
    X_important[:,i] = X[:,int(most_important_features[i][0])]

  return (X_important, importance_df)



In [ ]:
#PCA - Manipuluj wartością "n_components"

start = time.time()
pca = decomposition.PCA(n_components = 10, random_state = 2137, svd_solver="auto")
pca.fit(X,Y.ravel())
for i in range(len(pca.explained_variance_ratio_)):
  print("\nExplained Variance for PC{0:}: {1:.2f}%".format(i,pca.explained_variance_ratio_[i]*100))
X_pca = pca.transform(X)
print("\nPCA zajęło {} sekund".format(time.time() - start))

#MDS - Manipuluj wartościami "n_components", "dissimilarity" i "max_iter"

start = time.time()
mds = MDS(n_components = 10, random_state = 2137,
          dissimilarity = "euclidean", max_iter = 300)
X_mds = mds.fit_transform(X)
print("\nMDS zajęło {} sekund".format(time.time() - start))

#Decision Tree - Manipuluj wartością "n_estimators"

start = time.time()
tree_clf = ensemble.ExtraTreesClassifier(n_estimators = 1000,    
                                         random_state=2137)
tree_clf = tree_clf.fit(X, Y.ravel())
feature_model = feature_selection.SelectFromModel(tree_clf, prefit = True)
X_tree = feature_model.transform(X)
print("\nUtworzenie drzew zajęło {} sekund".format(time.time() - start))

#Infinite Feature Selection - Manipuluj wartościami n_features oraz alpha

start = time.time()
X_inf, Inf_df = InfiniteFeatureSelection(X, n_features = 10, alpha = 0.3)
print("\nInfinite Feature Selection zajęło {} sekund".format(time.time() - start))


### **Zadanie 4** 

Zastosuj funkcję `LogitModel` na danych otrzymanych w zadaniu trzecim i określ który model dał najlepsze wyniki oraz jaka może być tego przyczyna. Możesz testować pomysły z zadania 3. (punkt za rozumowanie badawcze i umiejętność wyciągania wniosków)

<b>Wyniki dla X (pełnowymiarowych danych)</b>

In [ ]:
results_X = LogitModel(X, Y)

<b> Wyniki dla X_pca </b>


In [ ]:
results_PCA = LogitModel(X_pca, Y)

<b>Wyniki dla X_mds</b>

In [ ]:
results_MDS = LogitModel(X_mds,Y)

<b>Wyniki dla X_tree</b>

In [ ]:
results_TREE = LogitModel(X_tree,Y)

<b> Wyniki dla X_inf</b>


In [ ]:
results_Inf = LogitModel(X_inf,Y)

### Zadanie 5 - Machine Learning na bogato // Playground

Rozwiąż powyższe zadanie używając biblioteki tensorflow z wysokopoziomowym API "Keras" - wypełnij luki w kodzie oraz dobierz najlepsze hiperparametry. Punktów nie ma ale też jest zaje...

In [ ]:
import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
print(tf.__version__)

In [ ]:
callback = [ModelCheckpoint("./best.h5", verbose = 1, save_best_only = True,
                            save_weights_only = True, mode = 'min')]

In [ ]:
input_dim = X_tree.shape[1]


model = Sequential()
model.add(Dense(100, activation = 'relu', input_dim = input_dim))
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer = "Adam", metrics = ["AUC"])
model.summary()

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_tree, Y,
                                                  test_size = 0.3,
                                                  shuffle = True,
                                                  random_state = 2137)
model.fit(X_train, Y_train, epochs = 500, validation_data=(X_val, Y_val),
          shuffle = True, callbacks = callback)

In [ ]:
model.load_weights("./best.h5")
model.evaluate(x = X_val, y= Y_val)

In [ ]:
tf.keras.backend.clear_session()